In [192]:
from selenium import webdriver
from selenium.webdriver.common.keys import Keys
from selenium.webdriver.common.by import By

import random
import numpy as np

In [106]:
moves_converter = {"\ue012": "left", "\ue013": "up", "\ue014": "right", "\ue015": "down"}

In [201]:
driver_path = "Users/davidesquer/Documents/chromedriverfolder/chromedrive"  #
browser = webdriver.Chrome(executable_path=driver_path) #abre el navegador
browser.get("https://2048.io") #abre el juego

game_element = browser.find_element(By.TAG_NAME, "body") #elemento para mandar los movimientos

# Define the possible moves
moves = [Keys.UP, Keys.RIGHT, Keys.DOWN, Keys.LEFT] #lista de movimientos posibles

score1 = browser.find_elements(By.XPATH, "/html/body/div/div[1]/div/div[1]") #elemento de score inicial
si = score1[0].text #texto de score inicial
si = si.split("\n")[0] #parsed score inicial

juego_total = [] #lista de listas que contiene el estado del juego, el movimiento, el score inicial y el score final para el juego completo

for i in range(200): #loop sobre el que se ejecuta el juego
    l1 = [] #lista que contiene el estado del juego actual
    tiles = browser.find_elements(By.CSS_SELECTOR, ".tile") #estado del juego actual
    for tile in tiles: #loop sobre el estado del juego actual
        l1.append(tile.get_attribute("class").split()) #agrega el estado del juego actual a la lista l1
        
    board = np.array([[0,0,0,0], [0,0,0,0], [0,0,0,0], [0,0,0,0]])
    for i in range(len(l1)):
        value = int(l1[i][1].split("-")[1])

        col = int(l1[i][2].split("-")[2])-1
        row = int(l1[i][2].split("-")[3])-1

        if board[row][col] < value:
            board[row][col] = value
    
    elements = browser.find_elements(By.CSS_SELECTOR, ".game-message") #elemento de mensaje de juego
    elements[0].text #texto de mensaje de juego

    if elements[0].text == "Game over!\nTry again": #si el juego termina, se rompe el loop
        break
    else:
        move = random.choice(moves) #elige un movimiento al azar
        game_element.send_keys(move) #ejecuta el movimiento elegido
    
    score = browser.find_elements(By.XPATH, "/html/body/div/div[1]/div/div[1]") #elemento de score final
    sf = score[0].text #texto de score final
    sf = sf.split("\n")[0] #parsed score final
    
    lista = [board, move, si, sf] #lista que contiene el estado del juego, el movimiento, el score inicial y el score final para el juego actual
    lista[1] = moves_converter[lista[1]] #convierte el movimiento a texto
    si = sf #actualiza el score inicial para el siguiente juego
    juego_total.append(lista) #agrega la lista del juego actual a la lista del juego completo

/var/folders/2g/ct_05dv51vbb20ykfhzzwxxm0000gn/T/ipykernel_29134/725578524.py:2: DeprecationWarning: executable_path has been deprecated, please pass in a Service object
  browser = webdriver.Chrome(executable_path=driver_path) #abre el navegador
